[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)


# Capstone Project: Mask Wearing Detector
---


## Table of Contents

1. Preprocessing
2. Modelling
---

In [1]:
# !pip install -U opencv-python-headless==4.5.2.52
 !pip install "opencv-python-headless<4.3"

     |████████████████████████████████| 38.2 MB 1.2 MB/s 


In [1]:
import os, shutil
# import cv2
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
os.mkdir("workspace")
os.mkdir("workspace/training")
os.mkdir("workspace/training/annotations")
os.mkdir("workspace/training/exported-models")
os.mkdir("workspace/training/images")
os.mkdir("workspace/training/images/train")
os.mkdir("workspace/training/images/test")
os.mkdir("workspace/training/models")
os.mkdir("workspace/training/pre-trained-models/")

In [3]:
#import data
!git clone https://github.com/siowcm/capstone.git
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'capstone'...
remote: Enumerating objects: 5821, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 5821 (delta 11), reused 30 (delta 10), pack-reused 5790
Receiving objects: 100% (5821/5821), 418.55 MiB | 34.61 MiB/s, done.
Resolving deltas: 100% (859/859), done.
Checking out files: 100% (5790/5790), done.
Cloning into 'models'...
remote: Enumerating objects: 69092, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 69092 (delta 51), reused 103 (delta 50), pack-reused 68979
Receiving objects: 100% (69092/69092), 577.28 MiB | 23.01 MiB/s, done.
Resolving deltas: 100% (48655/48655), done.
Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 15.33 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [4]:
#protobuf installation
%%bash
cd /content/models/research
pwd
protoc object_detection/protos/*.proto --python_out=.

/content/models/research


In [5]:
#COCO API installation
%%bash
cd cocoapi/PythonAPI
pwd
make
cp -r pycocotools /content/models/research

/content/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I../common -I/usr/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-m

../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
       ^~~
../common/maskApi.c:46:49: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
                                                 ^
../common/maskApi.c: In function ‘rleFrPoly’:
../common/maskApi.c:166:3: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
   ^~~
../common/maskApi.c:166:54: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
                                                      ^
../common/maskApi.c:167:3: warning: this ‘for’ clause does not guard... [-Wmisleading

In [6]:
#object detection API installation
%%bash
cd /content/models/research
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1686356 sha256=9ca5e99da62cdbb701d0574eb1382bb2b6b0c7fef4beb15619cc593e6de8641d
  Stored in directory: /tmp/pip-ephem-wheel-cache-eowayllo/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=ac4d538ceff1951a2efd17605781affc61e9b5dc326a0f00c3bfbd2119970a58
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=b186b4ea7d0cf3f5ccabf5059a1eb8b0aba895efdc990b11932ea20593f449cd
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=c57e5d3ec4f3162cdebbd9e9019e892a26f047f0e0d34f684d

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium=

In [7]:
#Test installation
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-02-15 00:41:17.334599: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0215 00:41:17.975443 140340260992896 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 2.39s
I0215 00:41:18.492544 140340260992896 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 2.39s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 1.11s
I0215 00:41:19.600804 140340260992896 test

In [8]:
# define path
IMGS_PATH = "capstone/input_data/images/"
ANNOT_PATH = "capstone/input_data/annotations/"
CROPPED_PATH = "capstone/output_data/cropped_images/"
CSV_PATH = "capstone/output_data/csv/"

In [ ]:
# !pip install -U tensorflow==2.7.0

In [9]:
df = pd.read_csv(CSV_PATH + "mask.csv")

In [11]:
# manually splitting the the data ensuring classes are evenly distributed and
# no data leakage

# saving the list of file names according to their classes
incorrect = list(
    set(df[df.label == "mask_weared_incorrect"]["images"].to_list())
)

without = list(set(df[df.label == "without_mask"]["images"].to_list()))

with_mask = list(set(df[df.label == "with_mask"]["images"].to_list()))

# ensuring that file names does not appear on other list
incorrect_u = incorrect
without_u = [i for i in without if i not in incorrect]
with_mask_U = [i for i in with_mask if i not in incorrect and i not in without]

# split data into train and test
test_incorrect = incorrect_u[int(0.8 * len(incorrect_u)) :]
train_incorrect = incorrect_u[: int(0.8 * len(incorrect_u))]

test_without = without_u[int(0.8 * len(without_u)) :]
train_without = without_u[: int(0.8 * len(without_u))]

test_with_mask = with_mask_U[int(0.8 * len(with_mask_U)) :]
train_with_mask = with_mask_U[: int(0.8 * len(with_mask_U))]

training_set = train_incorrect + train_without + train_with_mask
testing_set = test_incorrect + test_without + test_with_mask

In [12]:
# copy files into train and test folder
for file in training_set:
    source = IMGS_PATH + file
    destination = "workspace/training/images/train/"
    shutil.copy(source, destination)

for file in [i.replace(".png", ".xml") for i in training_set]:
    source = ANNOT_PATH + file
    destination = "workspace/training/images/train/"
    shutil.copy(source, destination)


for file in testing_set:
    source = IMGS_PATH + file
    destination = "workspace/training/images/test/"
    shutil.copy(source, destination)

for file in [i.replace(".png", ".xml") for i in testing_set]:
    source = ANNOT_PATH + file
    destination = "workspace/training/images/test/"
    shutil.copy(source, destination)

In [10]:
# generate label map
labels = [
    {"name": "with_mask", "id": 1},
    {"name": "without_mask", "id": 2},
    {"name": "mask_weared_incorrect", "id": 3},
]

with open("workspace/training/annotations/" + "label_map.pbtxt", "w") as f:
    for label in labels:
        f.write("item { \n")
        f.write("\tname:'{}'\n".format(label["name"]))
        f.write("\tid:{}\n".format(label["id"]))
        f.write("}\n")

In [12]:
shutil.copy(
    "capstone/input_data/annotations/maksssksksss0.xml",
    "workspace/training/images/train/",
)
shutil.copy(
    "capstone/input_data/images/maksssksksss0.png",
    "workspace/training/images/train/",
)

shutil.copy(
    "capstone/input_data/annotations/maksssksksss0.xml",
    "workspace/training/images/test/",
)
shutil.copy(
    "capstone/input_data/images/maksssksksss0.png",
    "workspace/training/images/test/",
)

'workspace/training/images/test/maksssksksss0.png'

In [14]:
#download script to generate tfrecord
os.mkdir("scripts")
os.mkdir("scripts/preprocessing")
!wget https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py -P scripts/preprocessing

--2022-02-15 00:49:52--  https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py
Resolving tensorflow-object-detection-api-tutorial.readthedocs.io (tensorflow-object-detection-api-tutorial.readthedocs.io)... 104.17.33.82, 104.17.32.82, 2606:4700::6811:2052, ...
Connecting to tensorflow-object-detection-api-tutorial.readthedocs.io (tensorflow-object-detection-api-tutorial.readthedocs.io)|104.17.33.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6410 (6.3K) [text/x-python]
Saving to: ‘scripts/preprocessing/generate_tfrecord.py’

generate_tfrecord.p 100%[===================>]   6.26K  --.-KB/s    in 0s      

2022-02-15 00:49:53 (70.2 MB/s) - ‘scripts/preprocessing/generate_tfrecord.py’ saved [6410/6410]



In [15]:
#create tfrecord
%%bash
cd scripts/preprocessing/
python generate_tfrecord.py -x ../../workspace/training/images/train -l ../../workspace/training/annotations/label_map.pbtxt -o ../../workspace/training/annotations/train.record
python generate_tfrecord.py -x ../../workspace/training/images/test -l ../../workspace/training/annotations/label_map.pbtxt -o ../../workspace/training/annotations/test.record  

Successfully created the TFRecord file: ../../workspace/training/annotations/train.record
Successfully created the TFRecord file: ../../workspace/training/annotations/test.record


In [16]:
#download pre-trained model
%%bash
cd workspace/training/pre-trained-models/
wget -nv http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz 
tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

2022-02-15 00:50:21 URL:http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz [244817203/244817203] -> "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz" [1]


In [17]:
'''
manually update pipeline.config or copy from github
=========
3 num_classes: 3
131 batch_size: 8
161 fine_tune_checkpoint: "workspace/training/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint" # Path to checkpoint of pre-trained model
172 182 label_map_path: "workspace/training/annotations/label_map.pbtxt" # Path to label map file
174 186 input_path: "workspace/training/annotations/train.record" # Path to training TFRecord file
'''

#copy pipline config

os.mkdir("workspace/training/models/my_ssd_resnet50_v1_fpn/")
shutil.copy(
    "capstone/code/pipeline.config",
    "workspace/training/models/my_ssd_resnet50_v1_fpn/"
)


'workspace/training/models/my_ssd_resnet50_v1_fpn/pipeline.config'

In [18]:
shutil.copy(
    "models/research/object_detection/model_main_tf2.py",
    "workspace/training/",
)

'workspace/training/model_main_tf2.py'

In [19]:
%%bash
cd workspace/training
python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

Traceback (most recent call last):
  File "model_main_tf2.py", line 32, in <module>
    from object_detection import model_lib_v2
  File "/usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py", line 29, in <module>
    from object_detection import eval_util
  File "/usr/local/lib/python3.7/dist-packages/object_detection/eval_util.py", line 36, in <module>
    from object_detection.metrics import lvis_evaluation
  File "/usr/local/lib/python3.7/dist-packages/object_detection/metrics/lvis_evaluation.py", line 23, in <module>
    from lvis import results as lvis_results
  File "/usr/local/lib/python3.7/dist-packages/lvis/__init__.py", line 5, in <module>
    from lvis.vis import LVISVis
  File "/usr/local/lib/python3.7/dist-packages/lvis/vis.py", line 1, in <module>
    import cv2
  File "/usr/local/lib/python3.7/dist-packages/cv2/__init__.py", line 9, in <module>
    from .cv2 import _registerMatType
ImportError: cannot import name '_registerMatType' from 'cv2.cv2' (/us

In [ ]:
!pip list